-----------------------------------------------------------------------------

<h1> MOFs CO2 working capacity prediction with XGBoost </h1>

------------------------------------------------------------

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.display import display
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Normalization
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

<h2>loading dataset</h2>

In [2]:
df = pd.read_csv('Datasets/cif_xyz_nonfg_train.csv');df.head(3)

,MOFname,volume,density,weight,surface_area,void_fraction,void_volume,functional_groups,metal_linker,organic_linker1,...,Lattice7,Lattice8,Lattice9,C+O+H,C,O,H,sumatom,mean_charge,sum_charge
0,mof_unit_2,2769.503842,1.326090,2211.697211,603.61,0.13794,0.1040,F-OMe,10,44,...,-0.600769,-3.410680,18.641128,173,71,35,67,194,5.150000e-09,0.000001
1,mof_unit_3,1089.818728,1.178856,773.687960,788.50,0.14874,0.1262,OMe-COOH,2,22,...,0.010529,0.148108,10.630959,80,32,14,34,82,-3.660000e-08,-0.000003
2,mof_unit_4,2205.198301,0.982408,1304.638720,1441.53,0.21814,0.2220,H-SO3H,9,17,...,-0.078915,-6.446998,18.164659,107,56,23,28,112,4.460000e-08,0.000005


seperate label column

In [3]:
df_label = df['CO2_working_capacity']
df = df.drop(['CO2_working_capacity','MOFname'],axis=1)
print(df.shape,' : ',df_label.shape);df.head(5)

(68584, 35)  :  (68584,)


,volume,density,weight,surface_area,void_fraction,void_volume,functional_groups,metal_linker,organic_linker1,organic_linker2,...,Lattice7,Lattice8,Lattice9,C+O+H,C,O,H,sumatom,mean_charge,sum_charge
0,2769.503842,1.326090,2211.697211,603.61,0.13794,0.1040,F-OMe,10,44,57,...,-0.600769,-3.410680,18.641128,173,71,35,67,194,5.150000e-09,0.000001
1,1089.818728,1.178856,773.687960,788.50,0.14874,0.1262,OMe-COOH,2,22,24,...,0.010529,0.148108,10.630959,80,32,14,34,82,-3.660000e-08,-0.000003
2,2205.198301,0.982408,1304.638720,1441.53,0.21814,0.2220,H-SO3H,9,17,24,...,-0.078915,-6.446998,18.164659,107,56,23,28,112,4.460000e-08,0.000005
3,3954.659761,0.647909,1543.027680,2430.55,0.37094,0.5725,Pr-NO2,9,7,23,...,-2.470500,-0.012234,22.470616,158,72,24,62,164,-1.830000e-08,-0.000003
4,3565.914939,0.910268,1954.749656,1530.02,0.33337,0.3662,NH2,10,53,55,...,0.012840,-4.025383,22.428988,165,68,28,69,182,2.200000e-08,0.000004


Normalizing functional groups

In [4]:
def check_in(pattern : str, loop : list or np.ndarray or None = df.functional_groups):
    return [int(pattern in str(x).split('-') )for x in loop ]

def count_in(pattern : str, loop : list or np.ndarray or None = df.functional_groups):
    return [str(x).count(pattern) for x in loop]

compounds = set(['SO3H','COOH','NH2','OH','CN','F','OMe','NHMe','NO2','Pr','Cl','OEt','Ph','Br','OPr','HCO','Et','Me','H','I'])
molecules = set(['N','O','C'])

func_data = {
    f'funccheck_{compound}':check_in(compound)
    for compound in compounds
}
func_data.update({
    f'funccount_{molecule}':count_in(molecule)
    for molecule in molecules
})
func_data.update({
    'num_func': [int('-' in str(x)) for x in df.functional_groups]
})

df.functional_groups = df.functional_groups.astype("category").cat.codes
func_df = pd.DataFrame(func_data);func_df


,funccheck_OEt,funccheck_HCO,funccheck_OMe,funccheck_Me,funccheck_NHMe,funccheck_Et,funccheck_CN,funccheck_H,funccheck_OH,funccheck_NO2,...,funccheck_I,funccheck_Pr,funccheck_F,funccheck_COOH,funccheck_SO3H,funccheck_OPr,funccount_C,funccount_N,funccount_O,num_func
0,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,1
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,3,1
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,1
3,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,1,1,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68579,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
68580,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
68581,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,1
68582,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0


Add more feature

In [5]:
df.insert(
    loc=0,
    column='difatom',
    value=df['sumatom']-df['C+O+H']
)
df.insert(
    loc=0,
    column='surface_to_volume',
    value=df['surface_area']/df['volume']
)
df.insert(
    loc=0,
    column='cubic_surface_area',
    value=((df['volume']**(1/3))**2)*6
)
df.void_volume = df.void_volume**2
df.surface_area = (df.surface_area**2)
df

,cubic_surface_area,surface_to_volume,difatom,volume,density,weight,surface_area,void_fraction,void_volume,functional_groups,...,Lattice7,Lattice8,Lattice9,C+O+H,C,O,H,sumatom,mean_charge,sum_charge
0,1183.275571,0.217949,21,2769.503842,1.326090,2211.697211,3.643450e+05,0.13794,0.010816,115,...,-0.600769,-3.410680,18.641128,173,71,35,67,194,5.150000e-09,0.000001
1,635.410072,0.723515,2,1089.818728,1.178856,773.687960,6.217322e+05,0.14874,0.015926,303,...,0.010529,0.148108,10.630959,80,32,14,34,82,-3.660000e-08,-0.000003
2,1016.520988,0.653696,5,2205.198301,0.982408,1304.638720,2.078009e+06,0.21814,0.049284,139,...,-0.078915,-6.446998,18.164659,107,56,23,28,112,4.460000e-08,0.000005
3,1500.458542,0.614604,6,3954.659761,0.647909,1543.027680,5.907573e+06,0.37094,0.327756,373,...,-2.470500,-0.012234,22.470616,158,72,24,62,164,-1.830000e-08,-0.000003
4,1400.442353,0.429068,17,3565.914939,0.910268,1954.749656,2.340961e+06,0.33337,0.134102,200,...,0.012840,-4.025383,22.428988,165,68,28,69,182,2.200000e-08,0.000004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68579,579.449866,0.173410,2,949.067112,1.318868,753.787520,2.708595e+04,0.00000,0.000000,360,...,4.211774,-0.032201,15.139554,84,38,8,38,86,-3.490000e-08,-0.000003
68580,678.367236,0.133957,4,1202.182553,1.440028,1042.538240,2.593427e+04,0.00000,0.000000,344,...,-0.709510,-0.700082,11.090143,100,56,8,36,104,3.850000e-08,0.000004
68581,673.135676,0.127061,9,1188.302573,1.399781,1001.700216,2.279719e+04,0.00000,0.000000,366,...,-0.398622,-0.155104,10.184892,110,47,8,55,119,3.360000e-08,0.000004
68582,788.548045,0.107124,11,1506.660363,1.645811,1493.296496,2.604999e+04,0.01108,0.000000,380,...,-2.389941,-1.681765,14.747027,115,38,27,50,126,-5.560000e-08,-0.000007


One-hotting metal/organic linkers and topology

In [6]:
metal_linker_int = df['metal_linker']-1
metal_one_hot = to_categorical(metal_linker_int,num_classes=12,dtype='int8')
metal_onehot_df = pd.DataFrame(metal_one_hot,columns=['ml_' + str(num) for num in range(1,13)])
print(metal_onehot_df.shape)
display(metal_onehot_df.head(2))
#---------------------------------------
org1_int = df['organic_linker1']-1
org1_one_hot = to_categorical(org1_int,num_classes=59,dtype='int8')
org1_onehot_df = pd.DataFrame(org1_one_hot,columns=['ol1_' + str(num) for num in range(1,60)])
print(org1_onehot_df.shape)
display(org1_onehot_df.head(2))
#--------------------------------------
org2_int = df['organic_linker2']-1
org2_one_hot = to_categorical(org2_int,num_classes=59,dtype='int8')
org2_onehot_df = pd.DataFrame(org2_one_hot,columns=['ol2_' + str(num) for num in range(1,60)])
print(org2_onehot_df.shape)
display(org2_onehot_df.head(2))
#--------------------------------------
top_int = df['topology']
top_one_hot = to_categorical(top_int,dtype='int8')
top_onehot_df = pd.DataFrame(top_one_hot,columns=['top_' + str(num) for num in range(0,11)])
print(top_onehot_df.shape)
display(top_onehot_df.head(2))

(68584, 12)


,ml_1,ml_2,ml_3,ml_4,ml_5,ml_6,ml_7,ml_8,ml_9,ml_10,ml_11,ml_12
0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0


(68584, 59)


,ol1_1,ol1_2,ol1_3,ol1_4,ol1_5,ol1_6,ol1_7,ol1_8,ol1_9,ol1_10,...,ol1_50,ol1_51,ol1_52,ol1_53,ol1_54,ol1_55,ol1_56,ol1_57,ol1_58,ol1_59
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


(68584, 59)


,ol2_1,ol2_2,ol2_3,ol2_4,ol2_5,ol2_6,ol2_7,ol2_8,ol2_9,ol2_10,...,ol2_50,ol2_51,ol2_52,ol2_53,ol2_54,ol2_55,ol2_56,ol2_57,ol2_58,ol2_59
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


(68584, 11)


,top_0,top_1,top_2,top_3,top_4,top_5,top_6,top_7,top_8,top_9,top_10
0,0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,0


<h1>Feature selection</h1>

In [7]:
df_col = dict(
    df_geometry_col = [
    'surface_area',
    'void_fraction',
    'density',
    'void_volume',
    'weight',
    'volume',
    ],
    df_function_col = [
    'functional_groups',
    'metal_linker',
    'organic_linker1',
    'organic_linker2',
    'topology',
    'CO2/N2_selectivity',
    'heat_adsorption_CO2_P0.15bar_T298K',
    ],
    df_cif_col = [
    'cell_length_a',
    'cell_length_b',
    'cell_length_c',
    'cell_angle_alpha',
    'cell_angle_beta',
    'cell_angle_gamma',
    'sum_charge',
    # 'mean_charge',
    ],
    df_xyz_col = [
    'Lattice1',
    # 'Lattice2',
    # 'Lattice3',
    'Lattice4',
    'Lattice5',
    # 'Lattice6', 
    'Lattice7',
    'Lattice8',
    'Lattice9',
    # 'C+O+H',
    'C',
    'O',
    'H',
    'sumatom',
    ],
    df_add_col = [
    'surface_to_volume',
    'difatom',
    'cubic_surface_area'
    ]
)

df_selected = df[sum(df_col.values(),[])]
print(df_selected.shape)
df_selected = pd.concat([df_selected,metal_onehot_df,org1_onehot_df,org2_onehot_df,top_onehot_df,func_df],axis=1)
print(df_selected.shape);df_selected.head()

(68584, 33)
(68584, 198)


,surface_area,void_fraction,density,void_volume,weight,volume,functional_groups,metal_linker,organic_linker1,organic_linker2,...,funccheck_I,funccheck_Pr,funccheck_F,funccheck_COOH,funccheck_SO3H,funccheck_OPr,funccount_C,funccount_N,funccount_O,num_func
0,3.643450e+05,0.13794,1.326090,0.010816,2211.697211,2769.503842,115,10,44,57,...,0,0,1,0,0,0,0,0,1,1
1,6.217322e+05,0.14874,1.178856,0.015926,773.687960,1089.818728,303,2,22,24,...,0,0,0,1,0,0,1,0,3,1
2,2.078009e+06,0.21814,0.982408,0.049284,1304.638720,2205.198301,139,9,17,24,...,0,0,0,0,1,0,0,0,1,1
3,5.907573e+06,0.37094,0.647909,0.327756,1543.027680,3954.659761,373,9,7,23,...,0,1,0,0,0,0,0,1,1,1
4,2.340961e+06,0.33337,0.910268,0.134102,1954.749656,3565.914939,200,10,53,55,...,0,0,0,0,0,0,0,1,0,0


<h2>convert to array</h2>

In [8]:
df_label_array = np.array(df_label)
df_array = np.array(df_selected)
f'{df_label_array.shape} : {df_array.shape}'

'(68584,) : (68584, 198)'

In [9]:
scaler = StandardScaler()
df_array = scaler.fit_transform(df_array)

In [10]:
df_reconstract = pd.DataFrame(df_array,columns=df_selected.columns);df_reconstract.head()

,surface_area,void_fraction,density,void_volume,weight,volume,functional_groups,metal_linker,organic_linker1,organic_linker2,...,funccheck_I,funccheck_Pr,funccheck_F,funccheck_COOH,funccheck_SO3H,funccheck_OPr,funccount_C,funccount_N,funccount_O,num_func
0,-0.691969,-0.736572,0.918729,-0.246631,0.440871,-0.140155,-0.719722,1.843899,2.975121,3.605251,...,-0.261572,-0.245045,3.007411,-0.310648,-0.304170,-0.294658,-0.666928,-0.681529,0.172547,0.604168
1,-0.650913,-0.670994,0.503256,-0.243108,-0.701450,-0.487715,0.987536,-0.700377,0.934865,0.338146,...,-0.261572,-0.245045,-0.332512,3.219082,-0.304170,-0.294658,1.231022,-0.681529,2.710928,0.604168
2,-0.418620,-0.249592,-0.051094,-0.220112,-0.279675,-0.256921,-0.501774,1.525864,0.471171,0.338146,...,-0.261572,-0.245045,-0.332512,-0.310648,3.287634,-0.294658,-0.666928,-0.681529,0.172547,0.604168
3,0.192240,0.678221,-0.995005,-0.028136,-0.090305,0.105077,1.623218,1.525864,-0.456218,0.239143,...,-0.261572,4.080875,-0.332512,-0.310648,-0.304170,-0.294658,-0.666928,1.206120,0.172547,0.604168
4,-0.376676,0.450093,-0.254662,-0.161639,0.236758,0.024638,0.052177,1.843899,3.809771,3.407244,...,-0.261572,-0.245045,-0.332512,-0.310648,-0.304170,-0.294658,-0.666928,1.206120,-1.096644,-1.655169


<h2>train-test split</h2>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_array,df_label_array,test_size=0.1,random_state=123)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.15,random_state=123)
print(f'train : {X_train.shape}\nval : {X_val.shape}\ntest : {X_test.shape}')

In [11]:
X_train, X_test, y_train, y_test = train_test_split(df_array,df_label_array,test_size=0.1,random_state=123)
print(f'train : {X_train.shape}\ntest : {X_test.shape}')

train : (61725, 198)
test : (6859, 198)


<h1> Model training </h1>

In [27]:
model = Sequential([
    Dense(64, input_shape=(198,),activation='relu'),
    Dropout(0.3),
    Dense(128,activation='relu'),
    Dropout(0.3),
    Dense(256,activation='relu'),
    Dropout(0.3),
    Dense(512,activation='relu'),
    Dropout(0.3),
    Dense(1),
])

In [28]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 64)                12736     
_________________________________________________________________
dropout_11 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 128)               8320      
_________________________________________________________________
dropout_12 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 256)               33024     
_________________________________________________________________
dropout_13 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 512)              

In [29]:
model.compile(
    optimizer='adam',
    loss='mae',
    metrics=['mae'],
    )

In [32]:
model.fit(
    X_train,
    y_train,
    epochs=500,
    batch_size= 1000,
    )

Epoch 1/500
62/62 [==============================] - 1s 9ms/step - loss: 20.2401 - mae: 20.2401
Epoch 2/500
62/62 [==============================] - 1s 10ms/step - loss: 20.1821 - mae: 20.1821
Epoch 3/500
62/62 [==============================] - 1s 9ms/step - loss: 20.2906 - mae: 20.2906
Epoch 4/500
62/62 [==============================] - 1s 9ms/step - loss: 20.2160 - mae: 20.2160
Epoch 5/500
62/62 [==============================] - 1s 9ms/step - loss: 20.2124 - mae: 20.2124
Epoch 6/500
62/62 [==============================] - 1s 9ms/step - loss: 20.4706 - mae: 20.4706
Epoch 7/500
62/62 [==============================] - 1s 10ms/step - loss: 20.2864 - mae: 20.2864
Epoch 8/500
62/62 [==============================] - 1s 9ms/step - loss: 20.2602 - mae: 20.2602
Epoch 9/500
62/62 [==============================] - 1s 9ms/step - loss: 20.1675 - mae: 20.1675
Epoch 10/500
62/62 [==============================] - 1s 9ms/step - loss: 20.3272 - mae: 20.3272
Epoch 11/500
62/62 [=================

In [33]:
pred = model.predict(X_test);pred = pred.reshape(6859);np.log10(mean_absolute_error(y_test,pred))

1.5156696823630662

In [ ]:
pred_df = pd.DataFrame([pred,y_test]).T;pred_df

In [ ]:
%matplotlib inline
plt.scatter(pred_df[0],pred_df[1]-(pred_df[0]));plt.show()

In [ ]:
plt.scatter(pred_df[0],pred_df[1]);plt.show()

<h1>Comparing</h1>

In [ ]:
pred_df_plot = pred_df[pred_df[0]>400]
tomuch = len(pred_df_plot[(pred_df_plot[1]-pred_df_plot[0]).astype(int)<0])
tolittle = len(pred_df_plot[(pred_df_plot[1]-pred_df_plot[0]).astype(int)>0])
equal = len(pred_df_plot[(pred_df_plot[1]-pred_df_plot[0]).astype(int)==0])
plt.bar(['tomuch','tolittle','equal'],[tomuch,tolittle,equal])

<h1>Evaluation</h1>

In [ ]:
testset = pd.read_csv('Datasets/cif_xyz_nonfg_test.csv')
print(testset.shape);testset.head()

In [ ]:
test_func_data = {
    f'funccheck_{compound}':check_in(compound,loop=testset.functional_groups)
    for compound in compounds
}
test_func_data.update({
    f'funccount_{molecule}':count_in(molecule,loop=testset.functional_groups)
    for molecule in molecules
})
test_func_data.update({
    'num_func': [int('-' in str(x)) for x in testset.functional_groups]
})
testset.functional_groups = testset.functional_groups.astype("category").cat.codes
test_func_df = pd.DataFrame(test_func_data);test_func_df.head(5)

In [ ]:
test_metal_linker_int = testset['metal_linker']-1
test_metal_one_hot = to_categorical(test_metal_linker_int,num_classes=12,dtype='int8')
test_metal_onehot_df = pd.DataFrame(test_metal_one_hot,columns=['ml_' + str(num) for num in range(1,13)])
display(test_metal_onehot_df.head(3))
#-------------------------------------------------
test_org1_int = testset['organic_linker1']-1
test_org1_one_hot = to_categorical(test_org1_int,num_classes=59,dtype='int8')
test_org1_onehot_df = pd.DataFrame(test_org1_one_hot,columns=['ol1_' + str(num) for num in range(1,60)])
display(test_org1_onehot_df.head(3))
#-------------------------------------------------
test_org2_int = testset['organic_linker2']-1
test_org2_one_hot = to_categorical(test_org2_int,num_classes=59,dtype='int8')
test_org2_onehot_df = pd.DataFrame(test_org2_one_hot,columns=['ol2_' + str(num) for num in range(1,60)])
display(test_org2_onehot_df.head(3))
#-------------------------------------------------
test_top_int = testset['topology']
test_top_one_hot = to_categorical(test_top_int,dtype='int8')
test_top_onehot_df = pd.DataFrame(test_top_one_hot,columns=['top_' + str(num) for num in range(0,11)])
display(test_top_onehot_df.head(3))

In [ ]:
testset.insert(
    loc=33,
    column='difatom',
    value=testset['sumatom']-testset['C+O+H']
)
testset.insert(
    loc=0,
    column='surface_to_volume',
    value=testset['surface_area']/testset['volume']
)
testset.insert(
    loc=0,
    column='cubic_surface_area',
    value=((testset['volume']**(1/3))**2)*6
)
testset.void_volume = testset.void_volume**2
testset.surface_area = (testset.surface_area**2)

In [ ]:
testset = testset[sum(df_col.values(),[])]
testset = pd.concat([testset,
test_metal_onehot_df,
test_org1_onehot_df,
test_org2_onehot_df,
test_top_onehot_df,
test_func_df
],axis=1);testset.head(3)

check validation

In [ ]:
print(f'{len(testset.columns)} : {len(df_selected.columns)}')
if(len(testset.columns)==len(df_selected.columns)):
    print(all(testset.columns == df_selected.columns))
    print(all(testset.dtypes == df_selected.dtypes))
else:print(False)

In [ ]:
testset_array = np.array(testset);testset.shape

predict

In [ ]:
test_pred = xg_reg1.predict(testset_array);test_pred

convert to dataframe

In [ ]:
submission = pd.DataFrame({
    "id": [str(i) for i in range(68614,85614)],
    "CO2_working_capacity [mL/g]":test_pred
    })
submission.head()

save csv

In [ ]:
submission.to_csv('submission.csv',index=False,float_format='%.7f')

--------------------------------------------------------------------------------------------